## Next Steps

🎉 **Congratulations!** You've completed Part I of the Vibes Investigation series.

**Continue your journey:**
- **[Part II: Planning with Louie](06-vibes-investigation-II-planning.ipynb)** - Learn how to use Louie for strategic planning, roadmapping, and structured problem-solving

**Other resources:**
- [Louie Documentation](https://louie.readthedocs.io)
- [PyGraphistry Documentation](https://pygraphistry.readthedocs.io)

## Welcome to Vibes Investigating with Louie!

**This is Part I of a two-part series on Vibes Investigation:**
- **Part I (this notebook)**: General investigation techniques and exploratory analysis
- **[Part II](06-vibes-investigation-II-planning.ipynb)**: Planning, roadmapping, and strategic analysis

This tutorial demonstrates how to use Louie for investigative workflows, exploring data relationships, and uncovering insights through natural language queries.

## 1. Install

Get package `louieai` using pip or uv. Louie requires Python 3.10+ and Pandas/Arrow

It installs PyGraphistry 0.41+ for auth & graphs

In [1]:
!pip install -q louieai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


## Import

In [2]:
import graphistry

import louieai

print("graphistry", graphistry.__version__, "louieai", louieai.__version__)

graphistry 0.41.0 louieai 0.5.6.dev1+g08b6fab.d20250804


## Configure

1. Graphistry server: Holds your account, semantic layer, and adds GPU viz & remote GFQL querying

2. Louie server: Datathreads & agentic vibes


In [3]:
import getpass
import os

LOUIE_SERVER_URL = os.environ.get("LOUIE_SERVER", "https://den.louie.ai")
GRAPHISTRY_DOMAIN = os.environ.get("GRAPHISTRY_SERVER", "hub.graphistry.com")

# Check for Graphistry personal key credentials
personal_key_id = os.environ.get("GRAPHISTRY_PERSONAL_KEY_ID")
personal_key_secret = os.environ.get("GRAPHISTRY_PERSONAL_KEY_SECRET")
org_name = os.environ.get("GRAPHISTRY_ORG_NAME")

if not personal_key_id or not personal_key_secret or not org_name:
    print("🔐 Graphistry Authentication Required")
    print("=" * 50)
    print("You need a Graphistry personal API key to run this notebook.")
    print("\nTo get credentials:")
    print("1. Sign up/login at https://hub.graphistry.com")
    print("2. Go to https://hub.graphistry.com/users/personal/key/")
    print("3. Create a new personal API key")
    print("=" * 50)
    print("\nPlease enter your credentials:")
    personal_key_id = input("Personal Key ID: ")
    personal_key_secret = getpass.getpass("Personal Key Secret: ")
    org_name = input("Organization Name: ")
    print("\n💡 Tip: For automated execution, set environment variables:")
    print("   export GRAPHISTRY_PERSONAL_KEY_ID='your_key_id'")
    print("   export GRAPHISTRY_PERSONAL_KEY_SECRET='your_key_secret'")
    print("   export GRAPHISTRY_ORG_NAME='your_org_name'")

# MAKE API KEY: https://hub.graphistry.com/users/personal/key/
g = graphistry.register(
    api=3,
    server=GRAPHISTRY_DOMAIN,
    personal_key_id=personal_key_id,
    personal_key_secret=personal_key_secret,
    org_name=org_name,
)


def new_thread(share_mode="Private"):
    return louieai(g, server_url=LOUIE_SERVER_URL, share_mode=share_mode)


lui = new_thread()

print("Connected!")

Connected!


## Hello World

Louie is an agent that uses many other LLMs and agents. You can start just as if it is ChatGPT chat.

In [4]:
lui("""

sing me a song

""")

<LouieAI Notebook Interface | Session: Active | History: 1 responses | Traces: Disabled | Latest: 1 text>

In [5]:
lui("""

sing it again, but now in pig latin!

""")

<LouieAI Notebook Interface | Session: Active | History: 2 responses | Traces: Disabled | Latest: 1 text>

In [6]:
lui(
    """

back to normal, but now with traces!

""",
    traces=True,
)

<LouieAI Notebook Interface | Session: Active | History: 3 responses | Traces: Disabled | Latest: 1 text>

## Use `lui` as a chat cursor

The `lui` object indexes into your chat (notebook) and its artifacts

You can make new chats, and add to existing

In [7]:
lui.thread_id

'D_GzyhX4FS0O2zDHUvNeak'

In [8]:
# Save old
lui_old = lui

# Fresh thread with empty history
lui = new_thread()

lui("sing me a new song")
lui("sing me another song")

lui.thread_id

'D_s1dIkJBSmUgDeMvvaMuJ'

In [9]:
# Latest text element in current run
lui.text

"🎶 I'm Louie, the data hound, in a cyberpunk town, chasing data trails, never backing down. With a bark and a wag, I dive into the code, solving puzzles, lightening the load. From queries to clusters, I make them align, turning data chaos into design. So let's dance through the data, you and me,"

In [10]:
# Latest text elements in its previous run
lui[-2].text

"🎶 Woof, woof, I'm Louie, the data dog, chasing bytes and bits, never lost in the fog. With a wag of my tail and a bark so bright, I'll fetch your data, day or night. From queries to graphs, I'll make it all clear, turning data puzzles into cheer. So let's dive into the data stream, together we'll make the perfect team"

In [11]:
# Latest text element in previous session
lui_old.text

"🎶 Woof woof, I'm Louie, the data dog, chasing data through the fog. With a wag of my tail and a bark so bright, I'll fetch your insights day and night. From queries to graphs, I'll make it clear, turning data puzzles into cheer. So let's dive in, let's explore, with Louie by your side, you'll never be bored! 🎶\n\n*Trace 1: Sniffing out the data trail, following every byte and detail.*\n*Trace 2: Analyzing patterns, connecting the dots, solving mysteries on the spot.*\n*Trace 3: With precision and speed, I track down the lead, ensuring your data needs are always freed.*\n\nLet's get back to solving"

## Agents

### Databricks SQL Passthrough

Louie has databases connected and preindexed as a managed semantic layer. You can query it quickly by doing pushdown queries that skim LLMs, so more like tool calls.


In [12]:
lui(
    """SELECT * FROM `client_demos`.`botsv3`.`o365_management_activity_flat_tcook`
    LIMIT 10""",
    agent="DatabricksPassthroughAgent",
)

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,None,SharePoint
4,104.238.59.42,0a627e9e-f0d0-6000-daf9-dc8f468313e3,2018-08-20T13:06:50,SharePoint,9b5ad97e-f03f-4648-a536-08d5f28d5cad,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
5,65.52.243.21,fb617e9e-c0c7-6000-32f0-2e462ff7bbde,2018-08-20T13:05:48,SharePoint,4b143e6d-91b0-4b07-7253-08d5f28d37c3,None,List,67091393-e290-421e-ac6a-2734e2b12a94,None,https://frothly-my.sharepoint.com/personal/pce...,...,None,None,None,ODMTADocCache/1.0,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
6,104.238.59.42,f9617e9e-f0de-6000-32f0-299be8e0a683,2018-08-20T13:05:40,SharePoint,a1cac3e4-3102-4e0c-d258-08d5f28d3331,No,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
7,104.238.59.42,f8617e9e-a0d4-6000-3667-381044fbe5d3,2018-08-20T13:05:36,SharePoint,7d6f73bd-f3c3-4c95-7b24-08d5f28d30de,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
8,199.66.91.253,08557e9e-2041-6000-daf9-d90e9f2c1023,2018-08-20T11:55:22,SharePoint,d7ee3d62-60cb-427b-b0db-08d5f26d99f1,No,File,67091393-e290-421e-ac6a-2734e2b12a94,57c5cd78-2a0f-42ee-8dab-afce305ec89e,https://frothly-my.sharepoint.com/personal/fyo...,...,tar,archive.tar,Documents,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
9,199.66.91.253,fc547e9e-f0c2-6000-1d75-39268176f5e6,2018-08-20T11:54:35,

In [13]:
## Quick reference of latest dataframe
lui.df.sample(3)

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
8,199.66.91.253,08557e9e-2041-6000-daf9-d90e9f2c1023,2018-08-20T11:55:22,SharePoint,d7ee3d62-60cb-427b-b0db-08d5f26d99f1,No,File,67091393-e290-421e-ac6a-2734e2b12a94,57c5cd78-2a0f-42ee-8dab-afce305ec89e,https://frothly-my.sharepoint.com/personal/fyo...,...,tar,archive.tar,Documents,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive


In [14]:
lui.df

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,None,SharePoint
4,104.238.59.42,0a627e9e-f0d0-6000-daf9-dc8f468313e3,2018-08-20T13:06:50,SharePoint,9b5ad97e-f03f-4648-a536-08d5f28d5cad,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
5,65.52.243.21,fb617e9e-c0c7-6000-32f0-2e462ff7bbde,2018-08-20T13:05:48,SharePoint,4b143e6d-91b0-4b07-7253-08d5f28d37c3,None,List,67091393-e290-421e-ac6a-2734e2b12a94,None,https://frothly-my.sharepoint.com/personal/pce...,...,None,None,None,ODMTADocCache/1.0,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
6,104.238.59.42,f9617e9e-f0de-6000-32f0-299be8e0a683,2018-08-20T13:05:40,SharePoint,a1cac3e4-3102-4e0c-d258-08d5f28d3331,No,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
7,104.238.59.42,f8617e9e-a0d4-6000-3667-381044fbe5d3,2018-08-20T13:05:36,SharePoint,7d6f73bd-f3c3-4c95-7b24-08d5f28d30de,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
8,199.66.91.253,08557e9e-2041-6000-daf9-d90e9f2c1023,2018-08-20T11:55:22,SharePoint,d7ee3d62-60cb-427b-b0db-08d5f26d99f1,No,File,67091393-e290-421e-ac6a-2734e2b12a94,57c5cd78-2a0f-42ee-8dab-afce305ec89e,https://frothly-my.sharepoint.com/personal/fyo...,...,tar,archive.tar,Documents,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
9,199.66.91.253,fc547e9e-f0c2-6000-1d75-39268176f5e6,2018-08-20T11:54:35,

### Databricks SQL AI

You can also talk to it using natural language via an agent

In [15]:
lui("get 4 events from o365_management_activity_flat_tcook", agent="DatabricksAgent")

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,None,SharePoint


### Louie Agent: Main Orchestrator Agent

Louie will pick appropriate agents and tools for you, so you do not need to specify them

In [16]:
lui("""get 4 events from databrick's o365_management_activity_flat_tcook""")

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,None,SharePoint


## Let's Vibe: Composing via DataFrames

In [17]:
lui("""

Get 100 events from databricks o365_management_activity_flat_tcook

""")

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0.0,1.0,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0.0,1.0,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0.0,1.0,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0.0,1.0,None,SharePoint
4,104.238.59.42,0a627e9e-f0d0-6000-daf9-dc8f468313e3,2018-08-20T13:06:50,SharePoint,9b5ad97e-f03f-4648-a536-08d5f28d5cad,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0.0,1.0,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,104.209.132.239,2fd04f66-77cc-415f-2155-08d5f269d0d3,2018-08-20T11:32:08,SharePoint,dab49c4d-7681-42d2-bba8-08d5f26a57eb,Yes,File,3f63cc5d-d2b8-4a9f-bc5f-255534837703,ef167dd3-18b5-4962-bbbe-bd0afc11c3eb,https://frothly-my.sharepoint.com/personal/jac...,...,NES,MAIN.NES,Documents/NESPack,ExportWorker,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0.0,1.0,0165abc4-c020-47fb-bb69-511981a12dbf,OneDrive
96,104.209.132.239,2fd04f66-77cc-415f-2155-08d5f269d0d3,2018-08-20T11:32:08,SharePoint,145f7516-0ebe-46b4-2db3-08d5f26a57a9,Yes,File,4a006be3-198c-4a2b-a996-62e64cdc9d7d,ed3d5448-a81e-498a-ad04-455ff8173228,https://frothly-my.sharepoint.com/personal/gho...,...,pdf,Belgian Lambic Blend 3278.pdf,Documents/Frothly-Shared/Yeasts,ExportWorker,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0.0,1.0,46ca8820-8a7b-46eb-b077-7d9024e32b39,OneDrive
97,104.209.132.239,2fd04f66-77cc-415f-2155-08d5f269d0d3,2018-08-20T11:32:08,SharePoint,f1a27022-f1ab-430e-f477-08d5f26a5790,Yes,File,4a006be3-198c-4a2b-a996-62e64cdc9d7d,e220cbc7-e981-406b-b4ac-7bbf18ed5c41,https://frothly-my.sharepoint.com/personal/gho...,...,pdf,37354-pdf.pdf,Documents/Frothly-Shared/Yeasts,ExportWorker,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0.0,1.0,46ca8820-8a7b-46eb-b077-7d9024e32b39,OneDrive
98,104.209.132.239,2fd04f66-77cc-415f-2155-08d5f269d0d3,2018-08-20T11:32:07,SharePoint,003f1764-fc13-4326-ecb0-08d5f26a570e,Yes,File,ac9249b5-51e5-42e7-b3f1-f0c6e4674046,e72bdf4d-f31c-4c93-aedc-6fa355ae3eff,https://frothly.sharepoint.com/sites/Frothly_S...,...,ods,Come Again Double Cream Ale.ods,Shared Documents/specia

In [18]:
df_id = next(e["id"] for e in reversed(lui.elements) if e.get("type") == "dataframe")
print("df_id", df_id)

lui(f"""Draw a graph using any column interesting for investigations from df {df_id}""")

<LouieAI Notebook Interface | Session: Active | History: 7 responses | Traces: Disabled | Latest: 3 text>

# Prompt Templates

### Reuse prompts as template files

Prompt template files and TODO/plan files are key to quickly using agents, having them do long runs, and increasing how much the AI manages itself.

Likewise, instead of manually editing plans, this helps enable you to tell the AI how to do that for you. As templates get more advanced, self-modifying plans become key.

In [19]:
lui_template = new_thread()

lui_template("Repeat this out: The best jokes are about 19th century scientists")

lui_template.thread_id

'D_SUNGJP8qPczYFtF6jl8M'

In [20]:
lui_joke = new_thread()

lui_joke(f"Read dthread {lui_template.thread_id}, and only after, make a joke about it")

<LouieAI Notebook Interface | Session: Active | History: 1 responses | Traces: Disabled | Latest: 3 text>

In [21]:
print(lui_joke.text)

The notebook with the ID "D_SUNGJP8qPczYFtF6jl8M" contains a humorous statement emphasizing that the best jokes are about 19th-century scientists. This suggests a playful engagement with historical scientific figures, possibly highlighting their quirks or contributions in a light-hearted manner. The content reflects a blend of humor and historical interest, which could appeal to those who enjoy both science and comedy.


## Next Steps

🎉 **Congratulations!** You've completed Part I of the Vibes Investigation series.

**Continue your journey:**
- **[Part II: Planning with Louie](06-vibes-investigation-II-planning.ipynb)** - Learn how to use Louie for strategic planning, roadmapping, and structured problem-solving

**Other resources:**
- [Louie Documentation](https://louie-py.readthedocs.io)
- [PyGraphistry Documentation](https://pygraphistry.readthedocs.io)